# CartPole-v0

> reference: 

A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. 

A `reward` of +1 is provided for every timestep that the pole remains upright. The episode ends when `the pole is more than 15 degrees from vertical`, or `the cart moves more than 2.4 units from the center`.

In [1]:
import gym
from collections import deque
import itertools
import numpy as np
import random
import tensorflow as tf

random.seed(42)

In [6]:
class A2CAgent():
    def __init__(self, state_size, action_size):
        self._state_size = state_size
        self._action_size = action_size        
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self._learning_rate = 0.001
        
        # build model
        self.value_size = 1# action_size # 1
        self.actor = self._build_actor()
        self.critic = self._build_critic()
        
        # tensorflow
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def _build_actor(my, hidden_size=24):
        class actorModel():
            def __init__(self):
                self.x = tf.placeholder(tf.float32, [None, my._state_size])
                nn = tf.layers.dense(self.x, hidden_size, activation=tf.nn.relu)
                nn = tf.layers.dense(nn, hidden_size, activation=tf.nn.relu)
                all_act = tf.layers.dense(nn, my._action_size)#, activation=tf.nn.linear)
                self.all_act_prob = tf.nn.softmax(all_act, name='act_prob')
        
                # op to sample an action
                self._sample = tf.reshape(tf.multinomial(self.all_act_prob, 1), [])

                self.advantages = tf.placeholder(tf.float32, [None, my._action_size]) 
                self.loss = -tf.reduce_sum(tf.multiply(self.all_act_prob, self.advantages))

                # update
                optimizer = tf.train.AdamOptimizer(my._learning_rate)
                self.train_op = optimizer.minimize(self.loss)
                
                # Summary for TensorBoard
#                 tf.summary.scalar('loss', self.loss)
#                 self.summary = tf.summary.merge_all()    # summary
                
        return actorModel()
    
    # critic: state is input and value of state is output of model
    def _build_critic(my, hidden_size=24):
        class criticModel():
            def __init__(self):
                self.x = tf.placeholder(tf.float32, [None, my._state_size])
                nn = tf.layers.dense(self.x, hidden_size, activation=tf.nn.relu)
                nn = tf.layers.dense(nn, hidden_size, activation=tf.nn.relu)
                self.Q = tf.layers.dense(nn, my.value_size)#, activation=tf.nn.linear)
        
                # Loss
                self.targetQ = tf.placeholder(tf.float32, [None, my.value_size])
                self.loss = tf.reduce_mean((self.Q - self.targetQ)**2)
                
                # Summary for TensorBoard
#                 tf.summary.scalar('loss', self.loss)
#                 self.summary = tf.summary.merge_all()    # summary
                
                # Training
                self.train_op = tf.train.AdamOptimizer(my._learning_rate).minimize(self.loss)   
                
        return criticModel()

    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

      
    # update policy network every episode
    def train_model(self, batch_size=20):
        def batch(iterable, n=1):
            l = len(iterable)
            for ndx in range(0, l, n):
                yield list(itertools.islice(iterable, ndx, min(ndx + n, l)))
                
                
        batch_data = list(batch(self.memory, batch_size))
        selected_batch = random.sample(batch_data, 1)
        
        for state, action, reward, next_state, done in selected_batch[0]:
            
            target = np.zeros((1, self.value_size))
            advantages = np.zeros((1, self._action_size))
            
            value = self.sess.run(self.critic.Q, feed_dict={self.critic.x: [state]}) # self.critic.predict(state)[0]
            next_value = self.sess.run(self.critic.Q, feed_dict={self.critic.x: [next_state]}) # self.critic.predict(next_state)[0]
            
            if done:
                advantages[0][action] = reward - value
                target[0][0] = reward
            else:
                advantages[0][action] = reward + self.gamma * (next_value) - value
                target[0][0] = reward + self.gamma * next_value
            
            feed_dict_actor = {
                self.actor.x: [state],
                self.actor.advantages: advantages,
            }
            feed_dict_critic = {
                self.critic.x: [state], 
                self.critic.targetQ: target,
            }
            _, actor_loss = self.sess.run([self.actor.train_op, self.actor.loss], feed_dict=feed_dict_actor)
#             summary = self.sess.run([self.actor.summary], feed_dict=feed_dict_actor)
            _, critic_loss = self.sess.run([self.critic.train_op, self.critic.loss], feed_dict=feed_dict_critic)
#             summary = self.sess.run([self.critic.summary], feed_dict=feed_dict_critic)

    # using the output of policy network, pick action stochastically
    def takeAction(self, state):
        return self.sess.run(self.actor._sample, feed_dict={self.actor.x: [state]})


In [7]:
episodes = 500
render = True
shows = 20
total_reward = list()

env = gym.make('CartPole-v0')
state = env.reset()
agent = A2CAgent(state.shape[0], env.action_space.n)

for e in range(episodes):
    state = env.reset()
    
    for time_t in range(500):
        
        if render:
            env.render()
        
        action = agent.takeAction(state)
        next_state, reward, done, _ = env.step(action)
        
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            total_reward.append(time_t)
            break
            
    agent.train_model(32)
    
    if e % shows == 0:
        print("Episode: {}, Total Reward: {}".format(e, np.mean(total_reward[-shows:])))
#         agent.update_target_model()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode: 0, Total Reward: 11.0
Episode: 20, Total Reward: 18.1
Episode: 40, Total Reward: 12.7
Episode: 60, Total Reward: 13.95
Episode: 80, Total Reward: 14.45
Episode: 100, Total Reward: 15.3
Episode: 120, Total Reward: 23.05
Episode: 140, Total Reward: 52.5
Episode: 160, Total Reward: 44.5
Episode: 180, Total Reward: 46.8
Episode: 200, Total Reward: 67.1
Episode: 220, Total Reward: 67.85
Episode: 240, Total Reward: 78.35
Episode: 260, Total Reward: 83.0
Episode: 280, Total Reward: 88.25
Episode: 300, Total Reward: 89.3
Episode: 320, Total Reward: 86.25
Episode: 340, Total Reward: 81.3
Episode: 360, Total Reward: 99.2
Episode: 380, Total Reward: 115.5
Episode: 400, Total Reward: 122.5
Episode: 420, Total Reward: 107.2
Episode: 440, Total Reward: 114.45
Episode: 460, Total Reward: 86.6
Episode: 480, Total Reward: 118.1
